In [26]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns 
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


# 1- Importing the dataset

In [27]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [28]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# 2- Exploring and understanding data

In [29]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


Looking at datatypes and null values in the table: **'Cabin'** has a lot of null values **'Age'** has some null values **'Embarked'** has only two null values

In [30]:
pd.pivot_table(train_data, index = 'Survived', values = ['Age','SibSp','Parch','Fare'])

,Age,Fare,Parch,SibSp
Survived,,,,
0,30.626179,22.117887,0.329690,0.553734
1,28.343690,48.395408,0.464912,0.473684


*(numerical values)* Comparing Survival rates across Age, SibSp, Parch, and Fare: Richer and younger people had higher tendencies of survival

In [31]:
print(pd.pivot_table(train_data, index = 'Survived', columns = 'Pclass', values = 'Ticket' ,aggfunc ='count'))
print()
print(pd.pivot_table(train_data, index = 'Survived', columns = 'Sex', values = 'Ticket' ,aggfunc ='count'))
print()
print(pd.pivot_table(train_data, index = 'Survived', columns = 'Embarked', values = 'Ticket' ,aggfunc ='count'))

Pclass      1   2    3
Survived              
0          80  97  372
1         136  87  119

Sex       female  male
Survived              
0             81   468
1            233   109

Embarked   C   Q    S
Survived             
0         75  47  427
1         93  30  217


*(categorical values)* Comparing rates accross Pclass, sex and Embarked: People at higer classes and females had higher tendencies of survival.

# 2- Exclude insignificant columns

In [32]:
train_data=train_data.drop(['PassengerId', 'Name','Ticket','Cabin'],axis=1)
test_data2=test_data.drop(['PassengerId', 'Name','Ticket','Cabin'],axis=1)

In [33]:
train_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [34]:
test_data2.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S


Observing missing data

In [35]:
train_data.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare          0
Embarked      2
dtype: int64

In [36]:
test_data2.isna().sum()

Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64

**Handling missing data**

* **Training data** 
* Imputing for Age (experimenting with replacing missing values with mean/median of column)
* Dropping for embark (2 rows only)
* **Testing data**  
* Imputing for Age (experimenting with replacing missing values with mean/median of column)
* Dropping for fare (1 row only)

In [37]:
#train_data.Age = train_data.Age.fillna(train_data.Age.mean())
train_data.Age = train_data.Age.fillna(train_data.Age.median())
train_data.dropna(subset=['Embarked'],inplace = True)

#test_data.Age = test_data.Age.fillna(train_data.Age.mean())
test_data2.Age = test_data2.Age.fillna(test_data2.Age.median())
test_data2.Fare = test_data2.Fare.fillna(test_data2.Age.median())

In [38]:
#check
train_data.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [39]:
#check
test_data2.isna().sum()

Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

# 4- Encoding categorical data

In [40]:
xtrain=train_data.iloc[:,1:]   
ytrain=train_data.iloc[:,0].values

xtest = test_data2.iloc[:, :]

* x has columns used to predict the dependent variable
* y is the dependent variable

In [41]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
column=ColumnTransformer(transformers=[('encoder', OneHotEncoder(), ['Sex','Embarked'])],remainder='passthrough')
xtrain=pd.DataFrame(column.fit_transform(xtrain))
xtest=pd.DataFrame(column.fit_transform(xtest))


# 5- Feature Scaling

In [42]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
#features to be scaled : age and fare
xtrain.iloc[:,6:10]= sc.fit_transform(xtrain.iloc[:,6:10])
xtest.iloc[:,6:10] = sc.transform(xtest.iloc[:,6:10])

# 6- Splitting dataset into testing data and training data

In [43]:
from sklearn.model_selection import train_test_split
xtrain, xtest1, ytrain, ytest = train_test_split (xtrain, ytrain, test_size=0.2)

# 7- Training the model

#  Random Forest Classifier

In [44]:
#For my model Random Forest Classifier
#To produce a more accurate result, it builds numerous decision trees and then merges them together.
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 20, criterion = 'entropy', random_state = 1)
classifier.fit(xtrain, ytrain)

RandomForestClassifier(criterion='entropy', n_estimators=20, random_state=1)

In [45]:
from sklearn.metrics import confusion_matrix, accuracy_score
ypredict = classifier.predict(xtest1)
ypredict
accuracy_score(ytest, ypredict)


0.8146067415730337

# Confusion Matrix

In [46]:
#Making the Confusion Matrix
confussionMatrix = confusion_matrix(ytest, ypredict)
print(confussionMatrix)
accuracy_score(ytest, ypredict)

[[89 12]
 [21 56]]


0.8146067415730337

In [47]:
predict = classifier.predict(xtest)

In [48]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predict})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
